In [ ]:
from matplotlib import pyplot as plt
from collections import defaultdict
import seaborn as sns
import pandas as pd
import psycopg2
import datetime
import pickle
import os

%load_ext nb_black

In [ ]:
def connect():
    conn = psycopg2.connect(
        user="postgres",
        password=os.environ.get("POSTGRES_PASS", ""),
        host="localhost",
        port=5432,
        database="venmo",
    )
    return conn


def location_to_state(location):
    name = location.raw["display_name"]
    if "United States of America" not in name:
        return None
    a, b, c = (["na"] + name.split(", "))[-3:]
    if b.replace("-", "").replace(":", "").isdigit():
        return a
    return b

In [ ]:
with open("user_id_to_loc.pkl", "rb") as f:
    user_id_to_loc_saved = pickle.load(f)
with open("geo_cache.pkl", "rb") as f:
    geo_cache = pickle.load(f)
with open("covid_words.pkl", "rb") as f:
    covid_words = pickle.load(f)

In [ ]:
transactions_by_state = defaultdict(list)

conn = connect()
cur = conn.cursor()

for user_id, (lat, lng, loc) in user_id_to_loc_saved.items():
    state = location_to_state(geo_cache[loc])
    if state is None:
        continue
    cur.execute(
        """
    SELECT id, message, type, created, actor_user_id, recipient_id FROM transactions 
      WHERE (recipient_id=%s OR actor_user_id=%s) AND created > '2020-01-01'
    """,
        (user_id, user_id),
    )
    transactions = cur.fetchall()
    transactions_by_state[state].extend(transactions)

print("Saving...")
with open("transactions_by_state.pkl", "wb") as f:
    pickle.dump(transactions_by_state, f)

conn.close()

In [ ]:
MODE = "2"

if MODE == "verified-only":

    with open("transactions_by_state.pkl", "rb") as f:
        transactions_by_state_saved = pickle.load(f)

    df_by_state_data = {"State": [], "Date": []}
    for state, transactions in transactions_by_state_saved.items():
        for id_, msg, type_, created, from_, to in transactions:
            df_by_state_data["State"].append(state)
            df_by_state_data["Date"].append(created.timestamp())
    df_by_state = pd.DataFrame(df_by_state_data)

else:

    with open("transactions_by_state_cluster.pkl", "rb") as f:
        transactions_by_state_saved = pickle.load(f)

    df_by_state_data = {"State": [], "Date": []}
    for state, transactions in transactions_by_state_saved.items():
        for created in transactions:
            df_by_state_data["State"].append(state)
            df_by_state_data["Date"].append(created.timestamp())
    df_by_state = pd.DataFrame(df_by_state_data)

In [ ]:
# As expected: big states, more transactions
df_by_state.groupby("State").count().sort_values(by="Date").iloc[-20:]

In [ ]:
# https://en.wikipedia.org/wiki/COVID-19_pandemic_lockdowns
LOCKDOWNS = {
    "California": (
        datetime.datetime(2020, 3, 19).timestamp(),
        datetime.datetime(2020, 5, 8).timestamp(),
    ),
    "Massachusetts": (
        datetime.datetime(2020, 3, 24).timestamp(),
        datetime.datetime(2020, 5, 4).timestamp(),
    ),
    "New York": (
        datetime.datetime(2020, 3, 22).timestamp(),
        datetime.datetime(2020, 6, 13).timestamp(),
    ),
    "Illinois": (
        datetime.datetime(2020, 3, 21).timestamp(),
        datetime.datetime(2020, 5, 30).timestamp(),
    ),
    "Michigan": (
        datetime.datetime(2020, 3, 24).timestamp(),
        datetime.datetime(2020, 4, 13).timestamp(),
    ),
}
STATE_TO_ABBR = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}


In [ ]:
CUT_OFF_DATE = 1601528400
BIN_CNT = 40
bins = list(
    pd.cut(df_by_state.Date.sort_values(), BIN_CNT)
    .apply(lambda x: x.left)
    .drop_duplicates()
)

In [ ]:
cases_df = pd.read_csv("United_States_COVID-19_Cases_and_Deaths_by_State_over_Time.csv")
cases_df["State"] = cases_df.state
cases_df["Date"] = cases_df.submission_date.apply(
    lambda date: datetime.datetime.strptime(date, "%m/%d/%Y").timestamp()
)
cases_df["Cases"] = cases_df.new_case.rolling(7).mean()


def plot_state(state, use_ax, copy_ax=None):

    state_abbr = STATE_TO_ABBR[state]

    df_state = df_by_state[df_by_state["State"] == state].copy()
    df_state["bin"] = pd.cut(df_state.Date, bins=bins)
    df_state["Transactions"] = 1
    df_state_hist = df_state[["bin", "Transactions"]].groupby("bin").sum().reset_index()
    df_state_hist["Date"] = df_state_hist.bin.apply(lambda x: x.mid)

    df_cases_state = cases_df[cases_df.state == state_abbr].copy()
    df_cases_state["bin"] = pd.cut(df_cases_state.Date, bins=bins)
    df_cases_state_hist = (
        df_cases_state[["bin", "new_case", "new_death"]]
        .groupby("bin")
        .sum()
        .reset_index()
    ).rename(columns={"new_case": "Cases", "new_death": "Deaths"})
    df_cases_state_hist["Date"] = df_cases_state_hist.bin.apply(lambda x: x.left)

    df_corr = df_cases_state_hist[["Cases"]].join(df_state_hist["Transactions"])
    corr = df_corr.corr().iloc[0, 1]

    sns.lineplot(
        data=df_state_hist[df_state_hist.bin.apply(lambda x: x.mid < CUT_OFF_DATE)],
        x="Date",
        y="Transactions",
        ax=use_ax,
        color="blue",
    ).set_title(
        "Venmo transactions in "
        + state
        + " (COVID cases in red, r = {})".format(round(corr, 2))
    )

    sns.lineplot(
        data=df_cases_state_hist.shift(2),
        x="Date",
        y="Cases",
        ax=use_ax.twinx(),
        color="red",
    )

    if state in LOCKDOWNS:
        lstart, lend = LOCKDOWNS[state]
        plt.axvline(x=lstart, linestyle="-")
        plt.axvline(x=lend, linestyle="--")

    use_ax.set_xticks(copy_ax.get_xticks())
    use_ax.set_xticklabels(copy_ax.get_xticklabels())
    use_ax.set_xticklabels(
        [
            datetime.datetime.fromtimestamp(ts).isoformat()[:10]
            for ts in use_ax.get_xticks()
        ],
        rotation=40,
    )


def plot_overall(use_ax, scatter_ax=None):
    df_state = df_by_state.copy()
    df_state["bin"] = pd.cut(df_state.Date, bins=bins)
    df_state["Transactions"] = 1
    df_state_hist = df_state[["bin", "Transactions"]].groupby("bin").sum().reset_index()
    df_state_hist["Date"] = df_state_hist.bin.apply(lambda x: x.mid)

    agg_cases = cases_df[["Date", "new_case"]].groupby("Date").sum()
    agg_cases["Cases"] = agg_cases.new_case.rolling(7).mean()
    agg_cases = agg_cases.dropna()

    df_cases_state = agg_cases
    df_cases_state["bin"] = pd.cut(df_cases_state.index, bins=bins)
    df_cases_state_hist = (
        df_cases_state[["bin", "new_case"]].groupby("bin").sum().reset_index()
    ).rename(columns={"new_case": "Cases"})
    df_cases_state_hist["Date"] = df_cases_state_hist.bin.apply(lambda x: x.left)

    corr = df_cases_state_hist["Cases"].corr(
        df_state_hist["Transactions"], method="pearson"
    )

    sns.lineplot(
        data=df_state_hist[df_state_hist.bin.apply(lambda x: x.mid < CUT_OFF_DATE)],
        x="Date",
        y="Transactions",
        ax=use_ax,
        color="blue",
    ).set_title(
        "Venmo transactions in USA (COVID cases in red, r = {})".format(round(corr, 2))
    )

    sns.lineplot(
        data=df_cases_state_hist.shift(2),
        x="Date",
        y="Cases",
        ax=use_ax.twinx(),
        color="red",
    )

    for state in LOCKDOWNS:
        lstart, lend = LOCKDOWNS[state]
        plt.axvline(x=lstart, linestyle="-")
        plt.axvline(x=lend, linestyle="--")

    use_ax.set_xticklabels(
        [
            datetime.datetime.fromtimestamp(ts).isoformat()[:10]
            for ts in use_ax.get_xticks()
        ],
        rotation=40,
    )


fig, ((ax, ax2), (ax3, ax4), (ax5, ax6), (ax7, ax8)) = plt.subplots(
    nrows=4, ncols=2, figsize=(15, 15)
)
fig.tight_layout(pad=8.0)
plot_overall(ax)
plot_state("California", ax2, copy_ax=ax)
plot_state("New York", ax4, copy_ax=ax)
plot_state("Texas", ax3, copy_ax=ax)
plot_state("New Jersey", ax5, copy_ax=ax)
plot_state("Massachusetts", ax7, copy_ax=ax)
plot_state("Michigan", ax6, copy_ax=ax)
plot_state("Illinois", ax8, copy_ax=ax)

In [ ]:
from shapely.geometry import Point
import geoplot.crs as gcrs
import geoplot as gplt
import geopandas as gpd


def get_state_corr(state):

    state_abbr = STATE_TO_ABBR[state]

    df_state = df_by_state[df_by_state["State"] == state].copy()
    df_state["bin"] = pd.cut(df_state.Date, bins=bins)
    df_state["Transactions"] = 1
    df_state_hist = df_state[["bin", "Transactions"]].groupby("bin").sum().reset_index()
    df_state_hist["Date"] = df_state_hist.bin.apply(lambda x: x.mid)

    df_cases_state = cases_df[cases_df.state == state_abbr].copy()
    df_cases_state["bin"] = pd.cut(df_cases_state.Date, bins=bins)
    df_cases_state_hist = (
        df_cases_state[["bin", "new_case", "new_death"]]
        .groupby("bin")
        .sum()
        .reset_index()
    ).rename(columns={"new_case": "Cases", "new_death": "Deaths"})
    df_cases_state_hist["Date"] = df_cases_state_hist.bin.apply(lambda x: x.mid)

    df_corr = df_cases_state_hist[["Cases"]].join(df_state_hist["Transactions"])
    return df_corr.corr().iloc[0, 1]


contiguous_usa = gpd.read_file(gplt.datasets.get_path("contiguous_usa"))
contiguous_usa["corr"] = contiguous_usa.state.apply(get_state_corr)
contiguous_usa = contiguous_usa.sort_values(by="corr")
contiguous_usa

In [ ]:
gplt.choropleth(
    contiguous_usa,
    hue="corr",
    projection=gcrs.AlbersEqualArea(),
    edgecolor="white",
    linewidth=1,
    cmap="Blues",
    legend=True,
    scheme="FisherJenks",
).set_title("Transactions vs COVID Cases Correlation")

In [ ]:
state_to_verified_users_found = defaultdict(list)
for _id, (_, _, loc) in user_id_to_loc_saved.items():
    state = location_to_state(geo_cache[loc])
    if state is None:
        continue
    state_to_verified_users_found[state].append(_id)
contiguous_usa["verified_users_found"] = contiguous_usa.state.apply(
    lambda s: len(state_to_verified_users_found.get(s, []))
)
usa_where_has_users = contiguous_usa[contiguous_usa.verified_users_found > 0].copy()
usa_where_has_users.corr()["population"]